In [1]:
import pandas as pd
import requests
import re
import os
import io
import requests
from nltk.tokenize import word_tokenize
from collections import Counter
import statsmodels.api as sm
from MyFunctions import html_to_text
import urllib.request
import gzip
import shutil
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 100)


In [2]:
urls = pd.read_csv('ng.txt', sep='|')
urls.head()
urls['adsh'] = urls['url'].str.split('/').str[7].str.split('-index').str[0]
date=urls['datadate'].astype(str)
cik=urls['cik'].astype(str)
urls['datadate']=date
urls['cik']=cik
urls.head(30)

,gvkey,cik,datadate,fyear,ngdum,ngeps,url,adsh
0,1004,1750,20040531,2003,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-04-018447-index.htm,0001104659-04-018447
1,1004,1750,20050531,2004,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-05-032222-index.htm,0001104659-05-032222
2,1004,1750,20060531,2005,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-06-046590-index.htm,0001104659-06-046590
3,1004,1750,20070531,2006,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-07-053342-index.htm,0001104659-07-053342
4,1004,1750,20090531,2008,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-09-043184-index.htm,0001104659-09-043184
5,1004,1750,20100531,2009,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-10-037891-index.htm,0001104659-10-037891
6,1004,1750,20120531,2011,1,0.45,https://www.sec.gov/Archives/edgar/data/1750/0001104659-12-049454-index.htm,0001104659-12-049454
7,1004,1750,20130531,2012,1,0.50,https://www.sec.gov/Archives/edgar/data/1750/0001104659-13-056679-index.htm,0001104659-13-056679
8,1004,1750,20140531,2013,0,NaN,https://www.sec.gov/Archives/edgar/data/1750/0001104659-14-051653-index.htm,0001104659-14-051653
9,1013,61478,20031031,2003,1,0.02,https://www.sec.gov/Archives/edgar/data/61478/0001157523-03-007030-index.htm,0001157523-03-007030


In [ ]:
def pr (df):
    pr=[]
    prurl=[]
    dcik=[]
    ddate=[]
    done = pd.DataFrame()
    for k in range(len(df)):

        # Extract the cik, adsh, date_filed, and index_url from the row
        cik = df['cik'][k]
        adsh = df['adsh'][k]
        date_filed = df['datadate'][k]
        index_url = df['url'][k]
        
     

        # Obtain the document name from the index table
        headers = {'User-Agent': 'University of Oklahoma jackwanghy@ou.edu'}
        html = requests.get(index_url, headers=headers).text 
        index_tables = pd.read_html(html)
        index_table = pd.DataFrame(index_tables[0])
        
        if str(index_table)=='None':
                # If there is no match, the match variable will be 'None'
                # so continue to the next row, skipping the rest of this loop
            continue
            
        document = index_table.loc[index_table.Type.str.contains('EX-99',na=False),'Document']
        if str(document)=='Series([], Name: Document, dtype: object)':
            document = index_table.loc[index_table.Type=='8-K','Document'].values[0]
        else:
            document = document.values[0]
                
        # Create the URL of the filing
        url = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+adsh.replace('-','')+'/'+document
        prurl.append(url)
        dcik.append(cik)
        ddate.append(date_filed)

        # Download the filing to the todirectory
        filing = requests.get(url,headers=headers).text

        # Convert html to text
        text = html_to_text(filing)
        pr.append(text)
        
        if k % 10000 == 0 and k != 0:
            done1=pd.DataFrame()
            done1['cik']=dcik
            done1['datadate']=ddate
            done1['pr']=pr
            done1.to_json('C:/Users/wang0169/Desktop/python/EPR/done1.json')
    
    
    done['cik']=dcik
    done['datadate']=ddate
    done['pr']=pr        
    return done           

In [ ]:
done.to_json('C:/Users/wang0169/Desktop/python/EPR/done.json')